In [1]:
import timeit
import numpy as np
import matplotlib.pyplot as plt
import pickle
from time import time

In [2]:
def init_kernel_bias(num_inp_channels, kernel_size, num_kernels,mean=0,std=0.01):
	shape = [num_inp_channels, kernel_size, kernel_size, num_kernels]
	weights = std*np.random.randn(*shape) + mean
	# weights/=np.sqrt(num_inp_channels)
	bias = std*np.random.randn(1,num_kernels) + mean
	return weights, bias

In [3]:
w0,b0=init_kernel_bias(num_inp_channels=32,kernel_size=3,num_kernels=128)

In [4]:
inp=np.random.randn(128,64,64,32)

In [44]:
def conv2d(self,inp,kernels,biases,stride=[1,1],padding=0):     #padding=(ksz-1)/2 for same shape in stride 1
    #inp[batches,row,col,d],kernels(d,ksz,ksz,num_ker),biases[1,num_ker],stride[row,col]
    inp=inp.transpose(0,3,1,2)  #inp[batches,d,row,col]
    output=[]
    ksz=kernels.shape[1]
    num_ker=kernels.shape[3]
    if not padding:                         #take care of padding in backprop too
        padding=(ksz-1)//2                  #currently don't give 'even' ksz
    out_row,out_col=((inp.shape[2]-ksz+2*padding)//stride[0]+1),((inp.shape[3]-ksz+2*padding)//stride[1]+1)
    batches,d,row,col=inp.shape
    row+=2*padding
    col+=2*padding
    padded=np.zeros((batches,d,row,col))
    padded[:,:,padding:-padding,padding:-padding]=inp
    # Take all windows into a matrix       # TO-DO: Cache up these calculations
    window=(np.arange(ksz)[:,None]*row+np.arange(ksz)).ravel()+np.arange(d)[:,None]*row*col
    slider=(np.arange(out_row*stride[0])[:,None]*row+np.arange(out_col*stride[1]))
    ind = window.ravel()+slider[::stride[0],::stride[1]].ravel()[:,None]
    kern = kernels.reshape(-1,num_ker)
    for img in padded:      #img[d,row,col]     #TODO: MAKE THIS FOR LOOP INTO SINGLE NUMPY OPERATION
        # windows(out_row*out_col, ksz*ksz*d) . kernels(d*ksz*ksz,num_ker)
        out=(np.dot(np.take(img, ind), kern))
        out=(out+biases).reshape(out_row,out_col,num_ker)
        output.append(out)
#     output=np.array([(np.dot(np.take(i,ind),kern)+biases) for i in padded]).reshape(batches,out_row,out_col,num_ker)
#     bind= np.arange(batches)[:,None]*d*row*col+ind.ravel()		#for batches
#     output=(np.dot(np.take(padded, bind).reshape(-1,d*ksz*ksz), kern)+biases).reshape(batches,out_row,out_col,num_ker)
    return np.array(output) #output[batches,out_row,out_col,num_ker]

In [49]:
%timeit conv0=conv2d(1,inp,w0,b0)

1.93 s ± 246 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [50]:
%timeit conv0=nn.conv2d(inp,w0,b0)

1.79 s ± 104 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [48]:
conv0=conv2d(1,inp,w0,b0)

In [54]:
%timeit d_X_inp,d_w0,d_b0=nn.conv2d_back(errors=conv0,inp=inp,kernels=w0,biases=b0)

6.34 s ± 28.4 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [49]:
conv0.shape

(10, 16, 16, 128)

In [5]:
#inp[batches,row,col,d],w0(d,ksz,ksz,num_ker),b0[1,num_ker],stride[row,col]
padding=0
stride=[1,1]
ipp=inp.transpose(0,3,1,2)  #ipp[batches,d,row,col]
output=[]
ksz=w0.shape[1]
num_ker=w0.shape[3]
if not padding: #take care of padding in backprop too
    padding=(ksz-1)//2  #currently don't give 'even' ksz
out_row,out_col=((ipp.shape[2]-ksz+2*padding)//stride[0]+1),((ipp.shape[3]-ksz+2*padding)//stride[1]+1)
batches,d,row,col=ipp.shape
row+=2*padding
col+=2*padding
padded=np.zeros((batches,d,row,col))
padded[:,:,padding:-padding,padding:-padding]=ipp

In [6]:
img=padded[0]

In [8]:
# %%timeit
window=(np.arange(ksz)[:,None]*row+np.arange(ksz)).ravel()+np.arange(d)[:,None]*row*col
slider=(np.arange(out_row*stride[0])[:,None]*row+np.arange(out_col*stride[1]))
ind = window.ravel()+slider[::stride[0],::stride[1]].ravel()[:,None]
# bind= np.arange(batches)[:,None]*d*row*col+ind.ravel()
kern = w0.reshape(-1,num_ker)
# output=(np.dot(np.take(padded, bind).reshape(-1,d*ksz*ksz), kern)).reshape(batches,out_row,out_col,num_ker)

In [9]:
padded.shape

(128, 32, 66, 66)

In [10]:
img.shape

(32, 66, 66)

In [11]:
kern.shape

(288, 128)

In [12]:
%timeit bind=np.arange(batches)[:,None]*d*row*col+ind.ravel()

191 ms ± 9.64 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


# Test Multiprocessing too nibba

In [47]:
from multiprocessing import Pool

In [44]:
%%timeit
output=np.empty((batches,out_row*out_col,num_ker))
for i,img in enumerate(padded):      #img[d,row,col]
    # windows(out_row*out_col, ksz*ksz*d) . kernels(d*ksz*ksz,num_ker)
    output[i]=np.dot(img.take(ind), kern)+b0
ans=output.reshape(batches,out_row,out_col,num_ker)

1.29 s ± 115 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [45]:
%%timeit
output=np.empty((batches,out_row*out_col,num_ker))
for i,img in enumerate(padded):      #img[d,row,col]
    # windows(out_row*out_col, ksz*ksz*d) . kernels(d*ksz*ksz,num_ker)
    output[i]=np.dot(img.take(ind), kern)
output+=b0
ans2=output.reshape(batches,out_row,out_col,num_ker)

1.21 s ± 13.2 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [95]:
def convolve(padded):
    batches=len(padded)
    output=np.empty((batches,out_row*out_col,num_ker))
    for i,img in enumerate(padded):      #img[d,row,col]
        # windows(out_row*out_col, ksz*ksz*d) . kernels(d*ksz*ksz,num_ker)
        output[i]=np.dot(img.take(ind), kern)
    output+=b0
    ans=output.reshape(batches,out_row,out_col,num_ker)
    return ans

In [98]:
%%timeit
cut=len(padded)//2
convolve(padded[:cut])

625 ms ± 12.2 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [111]:
Pool().starmap(convolve, [[padded[:cut]]])

KeyboardInterrupt: 

Traceback (most recent call last):
Process ForkPoolWorker-57:
Process ForkPoolWorker-58:
Process ForkPoolWorker-56:
  File "/usr/lib/python3.7/multiprocessing/process.py", line 297, in _bootstrap
    self.run()
Traceback (most recent call last):
Traceback (most recent call last):
  File "/usr/lib/python3.7/multiprocessing/process.py", line 297, in _bootstrap
    self.run()
  File "/usr/lib/python3.7/multiprocessing/process.py", line 297, in _bootstrap
    self.run()
  File "/usr/lib/python3.7/multiprocessing/process.py", line 99, in run
    self._target(*self._args, **self._kwargs)
  File "/usr/lib/python3.7/multiprocessing/process.py", line 99, in run
    self._target(*self._args, **self._kwargs)
  File "/usr/lib/python3.7/multiprocessing/process.py", line 99, in run
    self._target(*self._args, **self._kwargs)
  File "/usr/lib/python3.7/multiprocessing/pool.py", line 110, in worker
    task = get()
  File "/usr/lib/python3.7/multiprocessing/pool.py", line 110, in worker
    task = g

In [98]:
%timeit np.array([(np.dot(np.take(i,ind),kern)) for i in padded]).reshape(batches,out_row,out_col,num_ker)

1.39 s ± 80.9 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [95]:
%timeit (np.dot(np.take(padded, bind).reshape(-1,d*ksz*ksz), kern)).reshape(batches,out_row,out_col,num_ker)

928 ms ± 54.8 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [96]:
%timeit (np.dot(padded.take(bind).reshape(-1,d*ksz*ksz), kern)).reshape(batches,out_row,out_col,num_ker)

931 ms ± 20.4 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [23]:
np.take(img,ind).shape,kern.shape

((4096, 288), (288, 128))

In [24]:
(np.dot(np.take(img,ind), kern)).shape

(4096, 128)

In [26]:
ans=(np.dot(np.take(padded, bind).reshape(-1,d*ksz*ksz), kern)+b0).reshape(batches,out_row,out_col,num_ker)

In [27]:
ans.shape

(128, 64, 64, 128)

In [28]:
np.array([(np.dot(np.take(i,ind),kern)+b0) for i in padded]).reshape(batches,out_row,out_col,num_ker).__array_interface__

{'data': (139801886789648, False),
 'strides': None,
 'descr': [('', '<f8')],
 'typestr': '<f8',
 'shape': (128, 64, 64, 128),
 'version': 3}

In [29]:
np.array([(np.dot(np.take(i,ind),kern)+b0) for i in padded]).__array_interface__

{'data': (139801886789648, False),
 'strides': None,
 'descr': [('', '<f8')],
 'typestr': '<f8',
 'shape': (128, 4096, 128),
 'version': 3}

In [21]:
from numpy.fft  import fft2, ifft2
def np_fftconvolve(A, B):
    return np.real(ifft2(fft2(A)*fft2(B, s=A.shape)))
def test_numpy_fft(A, B, C, prefetching=False):
    if prefetching:
        for i_N in np.arange(A.shape[0]):
            for i_M in np.arange(B.shape[0]):
                C[i_N, i_M, :, :] = np_fftconvolve(A[i_N, :, :], B[i_M, :, :])
    else:
        for i_N in np.arange(A.shape[-1]):
            for i_M in np.arange(B.shape[-1]):
                C[:, :, i_N, i_M] = np_fftconvolve(A[:, :, i_N], B[:, :, i_M])

In [2]:
def func_int(a,b,c,d):
    return a+b+c+d
def func(a,b,c,d):
    return func_int,a,b,c,d

In [6]:
def add(fnc):
    print(fnc)

In [3]:
x=func(1,2,3,4)

In [4]:
x[0](*x[1:])

10

In [7]:
add(func(1,2,3,4))

(<function func_int at 0x7f4ea05859d8>, 1, 2, 3, 4)


In [5]:
x

(<function __main__.func_int(a, b, c, d)>, 1, 2, 3, 4)

In [2]:
import numpy as np

In [42]:
a=np.random.randint(0,9,(3,5,5))

In [49]:
b=(a>1)

In [50]:
b.sum()/b.size

0.7333333333333333

In [40]:
a.shape

(3, 5, 5)

In [1]:
import numpy as np

In [2]:
gg=np.random.random((8000,16,16,3))
np.random.shuffle(gg)

In [3]:
c=np.arange(64).reshape(16,4)

In [60]:
%timeit gg.shape[0]

95 ns ± 1.89 ns per loop (mean ± std. dev. of 7 runs, 10000000 loops each)


In [61]:
%timeit len(gg)

69.8 ns ± 1.6 ns per loop (mean ± std. dev. of 7 runs, 10000000 loops each)


In [4]:
# %%timeit
mn=gg.mean(axis=0)
st=gg.var(axis=0)

In [5]:
gg.shape,mn.shape,st.shape

((8000, 16, 16, 3), (16, 16, 3), (16, 16, 3))

In [6]:
xcap=((gg-mn)/st)

In [53]:
%timeit st=np.sqrt(((gg-mn)**2).mean(axis=0,keepdims=True))
%timeit st=gg.std(axis=0,keepdims=True)

21.8 ms ± 380 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)
21.6 ms ± 353 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [7]:
xcap.shape

(8000, 16, 16, 3)

In [14]:
%timeit gg*3

9.07 ms ± 204 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [15]:
%timeit 3*gg

8.88 ms ± 227 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [9]:
%timeit var=((gg-mn)**2).mean(axis=0)

In [19]:
%timeit 2*gg.sum(axis=0)

3.48 ms ± 40.1 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [18]:
%timeit (2*gg).sum(axis=0)

12.6 ms ± 167 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)
